In [1]:
import pandas as pd
import numpy as np 
import glob
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import ensemble
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.cluster import KMeans
from category_encoders import BinaryEncoder
import tensorflow as tf
from tensorflow import keras
from keras import metrics
import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold



2024-12-05 16:32:52.697136: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 16:32:52.860535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 16:32:52.901530: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 16:32:52.918029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 16:32:53.025299: I tensorflow/core/platform/cpu_feature_guar

In [2]:
os.system('clear') 


0

In [3]:
# Predata is used to make changes on the fly
predata = pd.read_csv("/home/eduardogh99/datasci207/talkin-ball-with-Santerre/training_data.csv")

/tmp/ipykernel_22548/3810129578.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  predata = pd.read_csv("/home/eduardogh99/datasci207/talkin-ball-with-Santerre/training_data.csv")


In [4]:
# Quick Check
rows = predata.shape[0]
cols = predata.shape[1]
print(f"Rows: {rows}")
print(f"Columns: {cols}")

Rows: 1068473
Columns: 167


In [5]:
# Quick Check
predata.head(5)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2024-03-02,13:32:53.75,1.0,1.0,"Quinn, JT",1.000234e+09,Right,OLE_REB,"Wilmes, Ben",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2024-03-02,13:33:10.55,1.0,2.0,"Quinn, JT",1.000234e+09,Right,OLE_REB,"Wilmes, Ben",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2024-03-02,13:33:23.01,1.0,3.0,"Quinn, JT",1.000234e+09,Right,OLE_REB,"Wilmes, Ben",...,NaN,NaN,High,High,High,Medium,Low,NaN,NaN,NaN
3,4,2024-03-02,13:33:44.31,1.0,4.0,"Quinn, JT",1.000234e+09,Right,OLE_REB,"Wilmes, Ben",...,NaN,NaN,High,High,High,High,Low,NaN,NaN,NaN
4,5,2024-03-02,13:34:08.96,1.0,5.0,"Quinn, JT",1.000234e+09,Right,OLE_REB,"Wilmes, Ben",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


In [6]:
# Copying Needed For Analysis
required_columns = ['TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB', 'TaggedHitType',
                    'PlayResult', 'RunsScored', 'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
                      'SpinRate', 'SpinAxis', 'Tilt', 'RelHeight', 'RelSide', 'Extension','InducedVertBreak', 
                      'HorzBreak', 'VertApprAngle', 'HorzApprAngle', 'Pitcher']
data = predata.dropna(subset=required_columns)
data = data[['TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB', 'TaggedHitType',
                    'PlayResult', 'RunsScored', 'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
                      'SpinRate', 'SpinAxis', 'Tilt', 'RelHeight', 'RelSide', 'Extension','InducedVertBreak', 
                      'HorzBreak', 'VertApprAngle', 'HorzApprAngle', 'Pitcher']]

rows = data.shape[0]
cols = data.shape[1]
print(f"Rows: {rows}")
print(f"Columns: {cols}")

Rows: 1051838
Columns: 22


In [7]:
# Checking Columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns


In [8]:
print(numerical_cols)
print(categorical_cols)

Index(['RunsScored', 'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
       'SpinRate', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension',
       'InducedVertBreak', 'HorzBreak', 'VertApprAngle', 'HorzApprAngle'],
      dtype='object')
Index(['TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB',
       'TaggedHitType', 'PlayResult', 'Tilt', 'Pitcher'],
      dtype='object')


In [9]:
data[categorical_cols].head(3)

,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,Tilt,Pitcher
0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,8:15,"Quinn, JT"
1,Fastball,Four-Seam,StrikeCalled,Undefined,Undefined,Undefined,12:15,"Quinn, JT"
2,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,12:00,"Quinn, JT"


In [10]:
# Using Binary Enocder to Encode while having memory low and not assuming ordinality
be = BinaryEncoder()
for col in categorical_cols:
    data = data.join(be.fit_transform(data[col]), how="outer")

In [11]:
data['PlayResult'].unique()

array(['Undefined', 'Single', 'Out', 'HomeRun', 'Double', 'Error',
       'Sacrifice', 'FieldersChoice', 'Triple', 'StolenBase',
       'CaughtStealing', 'error'], dtype=object)

In [12]:
data.drop(['PlayResult'], axis=1).columns
X = data

In [13]:
for col in categorical_cols:
    X = X.drop([col], axis=1)  # assuming RunsScored is your target

In [14]:
y = data['PlayResult']

In [15]:
le = LabelEncoder()
y =le.fit_transform(y)


In [16]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)

print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_val shape: {y_val.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (673176, 56)
X_val shape: (168294, 56)
X_test shape: (210368, 56)
y_train shape: (673176,)
y_val shape: (168294,)
y_test shape: (210368,)


In [18]:
scaler = StandardScaler()

#data[numerical_features] = scaler.fit_transform(data[numerical_features]) 
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [19]:
# Checking Columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [ ]:
clf = ensemble.RandomForestClassifier(n_estimators=7, n_jobs=1, random_state=7, max_depth=8)
# Fitting
clf.fit(X_train, y_train)

#Checking Scores
print("train acuracy: ", clf.score(X_train, y_train))
print("validation acuracy: ", clf.score(X_val, y_val))

train acuracy:  0.9989750080216764
validation acuracy:  0.9989720370304348


In [21]:
clf = ensemble.RandomForestClassifier(n_estimators=1, n_jobs=1, random_state=7, max_depth=1)
# Fitting
clf.fit(X_train, y_train)

#Checking Scores
print("train acuracy: ", clf.score(X_train, y_train))
print("validation acuracy: ", clf.score(X_val, y_val))

train acuracy:  0.9243139981223335
validation acuracy:  0.9255766693999786


In [ ]:
clf = ensemble.RandomForestClassifier(n_estimators=7, n_jobs=1, random_state=1234, max_depth=8)
# Fitting
clf.fit(X_train, y_train)

#Checking Scores
print("train acuracy: ", clf.score(X_train, y_train))
print("validation acuracy: ", clf.score(X_val, y_val))

train acuracy:  0.9974018681592927
validation acuracy:  0.9972369781453885


: 